In [ ]:
# Import harp-python for data interface and pandas for simple plotting
import harp
import pandas as pd 

# Load device reader
device = harp.create_reader("Data/Hobgoblin.harp")

# Load digital input and digital output
digital_output_set = device.DigitalOutputSet.read()
digital_output_clear = device.DigitalOutputClear.read()
digital_input_state = device.DigitalInputState.read()

# Inspect dataframe
print("LED onset time")
print(digital_output_set.head())
print("LED offset time")
print(digital_output_clear.head())
print("Button press time")
print(digital_input_state.head())

# Discard unused channels
digital_output_set = digital_output_set["GP15"]
digital_output_clear = digital_output_clear["GP15"]
digital_input_state = digital_input_state["GP2"]

# Keep button presses
digital_input_state = digital_input_state[digital_input_state == True]

# Extract valid responses (first button press that occurs within response_window/timeout)
response_window = 1.0
valid_response_times = []
for led_on in digital_output_set.index:
    for button_press in digital_input_state.index:
        response_time = button_press - led_on
        if 0 < response_time < response_window:
            valid_response_times.append(response_time)
            break

# Calculate and print hit/miss percentage
num_valid_responses = len(valid_response_times)
num_total_trials = len(digital_output_set.index)
hit_percentage = num_valid_responses / num_total_trials * 100
print(f"There were {num_valid_responses} valid responses out of {num_total_trials} trials, giving a hit rate of {hit_percentage}%")

# Plot valid response times
pd.Series(valid_response_times).plot(kind="hist")
